In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
import cv2
from tracker import Tracker 
from utils import select_bounding_box
from video_read_thread import FileVideoStream
import time
import imutils
from random import randint
from imutils.video import FPS
from people_recognize import PersonRecognizer


Using TensorFlow backend.


In [2]:
detection_model_path = 'haarcascade_files/haarcascade_frontalface_default.xml'
face_detection = cv2.CascadeClassifier(detection_model_path)
person_detector = PersonRecognizer()

def get_faces_from_haar(frame):
    bboxes=[]
    colors=[]
    frame=imutils.resize(frame,width=500)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    faces=face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(5,5),flags=cv2.CASCADE_SCALE_IMAGE)
    print(len(faces)," faces found in the frame")
    if len(faces)>0:
        for face in faces:
            (x,y,w,h) = faces[0]
            bboxes.append((x,y,w,h))
            frame2=frame.copy()
            frame2 = cv2.rectangle(frame2,(x,y),(x+w,y+h),(255,0,0),2)
            while cv2.waitKey(1) & 0xFF != ord('q'):
                cv2.imshow("Press q to continue",frame2)
    return bboxes,colors

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [3]:
def detect_people_from_frame(frame,recognizer=person_detector):
    height,width,channels=frame.shape
    boxes = recognizer.detect_people(frame)
    bboxes =[]
    colors = []
    for box in boxes :
        bboxes.append((int(box[-4]*width/512),int(box[-3]*height/512),int(box[-2]*width/512),int(box[-1])*height/512))
        colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    return bboxes,colors

In [15]:
# Set video to load
#videoPath = r"video/VID20200315124213.mp4"
videoPath = r"C:\Users\HP\PYT\Repos_dowloaded\people-counting-opencv\videos/example_01.mp4"
# Create a video capture object to read videos
cap=FileVideoStream(videoPath).start() 
# Read first frame
success, frame = cap.read()
# quit if unable to read the video file
#time.sleep(1.0)
#if not success:
#    print('Failed to read video')
#    sys.exit(1)


In [16]:
#print(bboxes)
#bboxes,colors=select_bounding_box(frame)
#bboxes,colors=get_faces_from_haar(frame)

fps = FPS().start()
fr = 30

# Process video and track objects
while cap.more():
    success, frame = cap.read()
    fr -=1 
    if fr == 0 :
        fr = 30
    if not success:
        break
    if fr == 30 :
        bboxes,colors=detect_people_from_frame(frame)
        multiTracker = Tracker(bboxes,frame).multiTracker
# get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)
# draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        #p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        p2 = (int( newbox[2]), int( newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
# show frame
    cv2.putText(frame,str(cap.qsize()),(30,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
    cv2.imshow('MultiTracker', frame)
# quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break
    fps.update()
cap.stop()
cv2.destroyAllWindows()

fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
Tracker Created ...
[INFO] elasped time: 97.95
[INFO] approx. FPS: 3.97


In [56]:
frame.shape

(720, 1280, 3)